# Семинар 1. Предобработка текстовых данных

В начале любой работы с текстом, как правило, требуется выполнить одни и теже действия: удалить все лишнее, разбить на предложения, токенизировать, нормализовать. На этом занятии мы разберем каждый из этих этапов. 

Если вы открыли тетрадку в Colab, нужно установить майстем и майморфи.

In [95]:
!pip install pymystem3==0.1.10
!pip install pymorphy2[fast]
!pip install razdel
!pip install gensim
!pip install nltk
!pip install rusenttokenize
!pip install regex

You should consider upgrading via the '/Users/mnefedov/.pyenv/versions/3.6.5/bin/python3.6 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mnefedov/.pyenv/versions/3.6.5/bin/python3.6 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mnefedov/.pyenv/versions/3.6.5/bin/python3.6 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mnefedov/.pyenv/versions/3.6.5/bin/python3.6 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mnefedov/.pyenv/versions/3.6.5/bin/python3.6 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mnefedov/.pyenv/versions/3.6.5/bin/python3.6 -m pip install --upgrade pip' command.
     |████████████████████████████████| 690 kB 2.5 MB/s eta 0:00:01
  Created wheel for regex: filename=regex-2020.10.23-cp36-cp36m-macosx_10_15_x86_64.whl size=282741 sha256=ec6d98c5c984073c795e9d74563b2c8c22045b0f140b7

In [125]:
# pymorphy2[fast] - быстрая версия пайморфи. Если у вас windows, то он вряд ли установится

In [47]:
# сразу импортируем все нужные библиотеки
# подробнее о каждой из них я расскажу по ходу 
import string
from gensim.utils import tokenize
from gensim.summarization.textcleaner import split_sentences
from rusenttokenize import ru_sent_tokenize
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import re, os, json
mystem = Mystem()
morph = MorphAnalyzer()
# если есть ошибки, доустановите библиотеки

## Удаление лишнего

Часто в данных, с которыми нам нужно работать помимо текста присутствует ещё какая-то лишняя информация: тэги, ссылки, код, разметка. Она, конечно, не всегда лишняя, но обычно от неё лучше избавиться.

Возьмем в качестве примера статью с Хабрахабра. Она скачана автоматически и в там остались некоторые тэги.

In [150]:
text = '''
Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!<br><br><img src="http://img172.imageshack.us/img172/8434/18274658kc4.png" alt="Сумасшедшее яйцо"><br><br><blockquote><h3>Что это такое?</h3><br>
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.<br>
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).<br><br><img src="https://habrastorage.org/getpro/habr/post_images/7c7/298/33c/7c729833cd942cc493e68833e3e0f12d.jpg" alt="Тепловое отслеживание популярности"><br></blockquote><br><br><a name="habracut"></a><br><br><blockquote><h3>Для кого это?</h3><br>
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн дает о себе знать — сервис массово популяризировался пол года-год назад. Относительно недорогой, но уж точно не из дешевых — сервис предполагает собой 4 платных линии и одну бесплатную. Дабы написать этот обзор я не поленился заплатить 19 долларов (в месяц) выбрав средний вариант — для нескольких проектов с включенными дополнительными функциями.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/0b0/0a0/b16/0b00a0b16b1eda28e35f39487dcd2545.jpg" alt="Отслеживание ссылок"></blockquote><br><blockquote><img src="https://habrastorage.org/getpro/habr/post_images/0b5/433/892/0b54338921665ffb5a90930147296f5b.jpg" alt="Список по популярности"><br><br>
Да-да, сервис не бесплатен. Точнее бесплатная возможность потестировать есть, но она немного обрезана (можно отслеживать только 5000 посетителей и всего 4 страницы на сайте (внимание — 4 страницы, а не сайтов), т.е. вполне хватает для того, чтобы понять полезность сервиса).</blockquote><br><blockquote><h3>Зачем это?</h3><br>
С помощью этого сервиса можно тасовать блоки на сайте, которые полезны пользователям больше всего. Больше не нужно спорить создателям — какой блок где расположить. В этом им поможет <a href="http://ttp://www.crazyegg.com" title="Сумасшедшие яйца">Сумасшедшее яйцо</a>.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/38c/af8/e75/38caf8e753782a01dc6419d3902edd57.jpg" alt="Добавление проекта"></blockquote><br><blockquote>Также этот сервис поможет вам понять — в какой зоне сайта лучше всего располагать рекламу, когда в вашем сервисе речь зайдет о монетизации. Ведь альтруизм это хорошо, а деньги на содержание сервиса нужны, и не лишним будет вычислить зоны где реклама будет приносить наибольшую отдачу, и наименьшее раздражение у пользователей.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/0dd/3fb/b34/0dd3fbb34709e77f3fbcd6523c7eac77.jpg" alt="еще полезности"></blockquote><br><blockquote><h3>Как это работает?</h3><br>
Никаких километровых скриптов вставлять не нужно, достаточно вставиь 2 строчки яваскрипта, и сервис начнет отслеживание. Насколько я понял — исполнительный скрипт работает на сервере <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>, поэтому ваш сайт от этого в производительности не потеряет ни секунды, а полезность довольно таки большая.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/a90/e4e/e67/a90e4ee6760978463e6306a2f5982e24.jpg" alt="управление популярностью"></blockquote><br><blockquote><h3>Сколько это стоит?</h3><br>
Как я уже упоминал — сервис далеко не бесплатен, хоть и имеет тестовую-бесплатную версию. Расскажу подробнее о тарификации.</blockquote><br><blockquote>1. Бесплатная тестовая версия. Включает в себя возможность отслеживания 5 000 посещений, 4 страницы на сайте.<br><br>
2. Базовая версия. В этой версии можно отследить 10 000 посещений, и 10 страниц, что вполне достаточно для среднего корпоративного сайта. Стоит базовый комплект — 9 долларов в месяц. В этот комплект включены все дополнительные функции.<br><br>
3. Версия «Стандарт». В неё входит возможность отслеживания 25 000 посещений на 20 страницах. Вполне подходит для тестирования нового стартапа. Стоит она 19 долларов в месяц, именно её я купил для тестирования сервиса, и написания этого обзора.<br><br>
4. Версия «Плюс». Отличается от предыдущей возможностью отслеживания 100 000 посещений, 50 страниц. Очень хороший тариф для крупных сервисов. Стоит 49 долларов в месяц. Довольно большие деньги за сервис, но они обычно с лихвой окупаются.<br><br>
5. Версия «Про». Стоит почти 100 долларов, имеет возможность отследить 250 000 посещений на ста страницах. Тариф подходит для монстров с большой посещаемостью и большим количеством страниц.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/f73/fb2/62d/f73fb262da4618a8dde67690cfd191ea.jpg" alt="Отслеживание статистики"></blockquote><br><blockquote><h3>Есть и аналоги</h3><br>
Я не поленился, и собрал еще пару ссылок с аналогами, которые предлагают такие же услуги, но немного дешевле.</blockquote><br><blockquote> 1. <a href="https://www.google.com/analytics/home/?hl=en" title="Шикарный сбор и анализ статистики">Google Analytics</a> — бесплатный сервис для сбора и анализа статистики, вывод статистики в наиболее наглядной форме, и без разнообразных рейтингов, счетчиков. Очень подробная и полезная вещь. Рекомендую, т.к. сам пользуюсь им для этого блога.<br><br>
2. <a href="http://www.mapsurface.com/" title="Сервис для отслеживания популярности блоков на сайте">MapSurface</a> — сам еще не использовал (т.к. предпочел CrazyEgg), но врядли будучи плохим сервис собрал бы множество положительных отзывов. К сожалению он сейчас находится в статусе закрытой беты.</blockquote><br><blockquote><h3>Вывод</h3><br>
Использовать можно, и нужно. Вот только тарифы довольно больно кусаются, но обычно эти деньги потом с лихвой отбиваются на повышении конвертации посетителей в деньги. Использовать сервис нужно для тестирования рекламных мест и удобства отдельных страниц, что помогает опять же повысить конвертацию. В общем и целом — <a href="http://ttp://www.crazyegg.com">полезная вещь</a> для каждого владельца сайтов, а для юзабилиста вообще практически обязательна. К счастью для людей, которые поиздержались деньгами в этом месяце — есть полезные аналоги.<br><br><img src="http://img241.imageshack.us/img241/2346/confettitnml5.jpg" alt="Интересная идея визуализации популярности - конфети"></blockquote><br><br>
Автор: <a href="http://www.birzool.com/" title="Я пишу о юзабилити веб интерфейсов">Ярослав Бирзул</a> (DezmASter).<br>
Источник: <a href="http://www.birzool.com/crazyegg/" title="Сумасшедшие яйца, первоисточник">Блог о юзабилити веб интерфейсов</a>.<br><br>
PS: Всех с прошедшим Новым годом! От всей души желаю вам всего самого-самого лучшего, чего вы желаете только в самых сокровенных мечтах. Удачно вам провести время.
'''

В html все тэги заключаются в фигурные скобки. Мы можем испьзовать это, чтобы легко избавиться от всех тэгов сразу. Напишем регулярное выражение, которое будет захватывать всё, что попадает между символами < и >, и не является '>'.

In [4]:
#re - модуль регулярных выражений в питоне
# функция sub заменяет все, что подходит под шаблон, на указанный текст
def remove_tags_1(text):
    return re.sub(r'<[^>]+>', '', text)

Проверим как работает наша функция.

In [5]:
print(remove_tags_1(text))


Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!Что это такое?
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).Для кого это?
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн да

Можно заметить, что в некоторых местах удаление тэгов приводит к тому, что между точкой и началом следующего 
предложения нет пробела, а это может помешать правильно токенизировать текст,
поэтому сделаем так, чтобы тэг заменялся пробелом

In [6]:
def remove_tags_2(text):
    return re.sub(r'<[^>]+>', ' ', text)

In [7]:
print(remove_tags_2(text))


Сегодняшняя заметка будет о сервисе отслеживания активности пользователя —  CrazyEgg . Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!       Что это такое?  
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов. 
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).             Для кого это?  
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный инт

Теперь куски текста не слепаются, но появились последовательности из нескольких пробелов, чтобы убрать их добавим ещё одно регулярное выражение и применим его к тексту, из которого уже удалили тэги

In [8]:
def remove_tags_3(text):
    no_tags_text = re.sub(r'<[^>]+>', ' ', text)
    no_space_sequences_text = re.sub('  +', ' ', no_tags_text)
    return no_space_sequences_text

In [9]:
print(remove_tags_3(text))


Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg . Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично! Что это такое? 
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов. 
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые). Для кого это? 
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший диз

Теперь текст более менее чистый.

In [10]:
text = remove_tags_3(text)

Говоря о регуляках, стоит еще упомянуть библиотеку - [regex](https://pypi.org/project/regex/) Это можно сказать re 2.0 - тут есть все, что есть в re, но есть и много дополнительных штук. Вот несколько полезных фичей regex:

In [96]:
import regex

In [ ]:
# нечеткий поиск

In [165]:
regex.findall('(конфетти){e<=1}', text)

['конфети', 'конфети']

In [194]:
[word for word in text.split() if regex.match('в(ерс){i<=3}', word)]

['версию.', 'версия.', 'версия.', 'версии']

In [192]:
[word for word in text.split() if regex.fullmatch('т(ест){i<=10}', word)]

['тестовая', 'тестирования', 'тестирования', 'тестирования']

In [ ]:
# сплиттер генератор

In [143]:
tokens = regex.splititer(' +', text)

In [146]:
next(tokens)

'the'

In [ ]:
# варианты списком

In [156]:
option_set = ["нет", "не", "он", 'она']
p = regex.compile(r"\L<options>", options=option_set)

In [157]:
p.findall(text)[:10]

['не', 'не', 'он', 'не', 'она', 'не', 'он', 'не', 'не', 'не']

Очистка текста достаточно индивидуальна и каждый раз приходится писать какие-то новые правила. А вот разбиение на предложения, токенизация и лемматизация - на 90% универсальны, т.е. одни и теже решения будут работать во всех задачах. К тому же, есть уже хорошие готовые инструменты для всего этого. 

## Разбиение на предложения

В __nltk__ есть уже готовая функция для разбивки на предложения. 

In [13]:
sent_tokenize(text, 'russian')[:10]

['\nСегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg .',
 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.',
 'Запоминается?',
 'Отлично!',
 'Что это такое?',
 'Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.',
 'Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).',
 'Для кого это?',
 'Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — инту

В __gensim__ тоже есть готовая функция

In [14]:
# это ещё и генератор, т.е. сразу подходит для больших корпусов
list(split_sentences(text))[:5]

['Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg .',
 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.',
 'Запоминается?',
 'Отлично!',
 'Что это такое?']

У DeepPavlov есть библиотека [**rusenttokenizer**](https://github.com/deepmipt/ru_sentence_tokenizer).

In [18]:
ru_sent_tokenize(text)[:10]

['Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg .',
 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.',
 'Запоминается?',
 'Отлично!',
 'Что это такое?',
 'Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.',
 'Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).',
 'Для кого это?',
 'Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуит

В проекте Natasha есть библиотека [**razdel**](https://github.com/natasha/razdel). Она чуть более навороченная.

In [25]:
sents = list(sentenize(text))

In [30]:
# числа тут - это спаны, индексы начала и конца предложения в изначальном тексте
sents[:10]

[Substring(1,
           86,
           'Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg .'),
 Substring(87,
           166,
           'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.'),
 Substring(167, 180, 'Запоминается?'),
 Substring(181, 189, 'Отлично!'),
 Substring(190, 204, 'Что это такое?'),
 Substring(206,
           399,
           'Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.'),
 Substring(401,
           815,
           'Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наибо

In [37]:
# у объекта Substring есть атрибуты start, stop и text. С помощью них можно вытащить нужное
[sent.text for sent in sents[:10]]

['Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg .',
 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.',
 'Запоминается?',
 'Отлично!',
 'Что это такое?',
 'Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.',
 'Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).',
 'Для кого это?',
 'Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуит

Если все-таки нужно добавить каких-то специфичных правил разбиения на предложения, можно опять же восползоваться регулярными выражениями. Однако в этом случае регулярка будет посложнее. 

Для разбиения текста по какому-то определенному разделителю подходит функция re.split. Давайте посмотрим, что получится, если в качестве разделителя использовать !?. пробел и заглавную букву.  

In [11]:
re.split('[!?\.] [А-Я]', text)[:10]

['\nСегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg ',
 ' не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо',
 'апоминается',
 'тлично',
 'то это такое? \nКак уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов. \nСервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые)',
 'ля кого это? \nРазумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфей

Проблема в том, что сам разделитель удаляется тоже, а нам нужно удалить только пробел между знаками препинания и заглавной буквой.

Решается эта проблема с помощью __look ahead__ и __look behind__ (название функционала в регулярных выражениях).  
Синтаксис там такой:  
**(?<=pattern)** положительное look-behind условие  
**(?<!pattern)** отрицательное look-behind условие  
**(?=pattern)** положительное look-ahead условие  
**(?!pattern)** отрицательное look-ahead условие  

Подробно про это написано тут: https://www.regular-expressions.info/lookaround.html  
А совсем подробно про (?...) выражения вообще вот тут - https://www.rexegg.com/regex-disambiguation.html

Look behind и look ahead превращают паттерн в условный, то есть проверяется есть ли он (до или после, соответственно), но его захвата не происходит. 

Обернём наше регулярное выражение и посмотрим, что получается:

In [12]:
re.split('(?<=[\.?!]) +(?=[А-ЯЁ])', text.replace('\n', ' '))[:10]

[' Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg .',
 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.',
 'Запоминается?',
 'Отлично!',
 'Что это такое?',
 'Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.',
 'Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).',
 'Для кого это?',
 'Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуи

## Токенизация

Мы разбили текст на предложения. Теперь предложения нужно разбить на токены. Под токенами обычно понимаются слова, но это могут быть и какие-то более длинные или короткие куски. 

Самый простой способ токенизации -- стандартный питоновский __str.split__ метод.
    По умолчанию он разбивает текст по последовательностям пробелом 
 (т.е. даже со второй версией remove_tags всё бы хорошо разделилось)

In [43]:
'1  2 3'.split(' ') # NB! .split() и .split(' ') - не одно и тоже

['1', '', '2', '3']

In [39]:
'1  2 3'.split()

['1', '2', '3']

In [1140]:
text.split()[:20]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 '—',
 'CrazyEgg',
 '.',
 'Я',
 'не',
 'знаю',
 'кому',
 'обязан',
 'сервис',
 'таким',
 'говорящим',
 'именем,']

Большая часть слов отделяется, но знаки препинания лепятся к словам.
Можно пройтись по всем словам и убрать из них пунктцацию с методом str.strip.

In [1124]:
#основные знаки преминания хранятся в питоноском модуле string.punctuation
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [46]:
# в этом списке не хватает кавычек-ёлочек, лапок, длинного тире и многоточия
string.punctuation += '«»—…“”'

In [119]:
[word.strip(string.punctuation) for word in text.split()][:10]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 '',
 'CrazyEgg']

Так не будут удаляться дефисы и точке в сокращениях, не разделенных пробелом.

In [150]:
'как-нибудь'.strip(punctuation)

'как-нибудь'

In [48]:
'т.е.'.strip(punctuation)

'т.е'

Ещё слова можно извлечь с помощью простого регулярного выражения:

In [118]:
re.findall('\w+', text)[:10]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 'CrazyEgg',
 'Я']

Как-нибудь так разделится, но это все-таки не так страшно.

Ещё есть готовые токенизаторы из nltk. Они не удаляют пунктуацию, а выделяют её отдельным токеном.

Например **wordpunct_tokenizer** разбирает по регулярке - *'\w+|[^\w\s]+'* 

In [117]:
wordpunct_tokenize(text)[:10]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 '—',
 'CrazyEgg']

Ещё есть **word_tokenize**. Он также построен на регулярках, но они там более сложные (учитывается последовательность некоторых 
символов, символы начала, конца слова и т.д). 

Специально подобранного под русский язык токенизатора там нет, 
но и с английским всё работает достаточно хорошо --
сокращения типа т.к собираются в один токен, дефисные слова тоже не разделяются, многоточия тут тоже не отделяются, но это можно поправить.

In [44]:
word_tokenize(text)[:10]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 '—',
 'CrazyEgg']

В генсиме тоже есть функция для токенизации

In [116]:
# опять же, это генератор
list(tokenize(text, lowercase=True))[:10]

['сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 'crazyegg',
 'я']

И в razdel тоже есть токенизация

In [50]:
list(razdel_tokenize(text))[:10]

[Substring(1, 12, 'Сегодняшняя'),
 Substring(13, 20, 'заметка'),
 Substring(21, 26, 'будет'),
 Substring(27, 28, 'о'),
 Substring(29, 36, 'сервисе'),
 Substring(37, 49, 'отслеживания'),
 Substring(50, 60, 'активности'),
 Substring(61, 73, 'пользователя'),
 Substring(74, 75, '—'),
 Substring(76, 84, 'CrazyEgg')]

In [51]:
[token.text for token in list(razdel_tokenize(text))[:10]]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 '—',
 'CrazyEgg']

# Нормализация

Полученные токены тоже чаще всего нужно привести к какому-то стандартному виду.

Самое простое и очевидное - привести всё к одному регистру:

In [46]:
'СловО'.lower()
# если не нужно разбивать на предложения, то это можно сделать в самом начале
# text.lower()

'слово'

Для языков со слабым словоизменением этого может быть достаточно. Но для русского с его склонениями и спряжениями лучше использовать стемминг или лемматизацию.

## Стемминг

Стемминг - это урезание слова до его "основы" (стема), т.е. такой части, которая является общей для всех словоформ в парадигме слова. По крайней мере так в теории. На практике стемминг сводится к отбрасыванию частотных окончаний.

Самый известный стеммер - стеммер Портера (или snowball стеммер). 
Подробнее про стеммер Портера можно почитать вот тут - <https://medium.com/@eigenein/стеммер-портера-для-русского-языка-d41c38b2d340>  
А совсем подробнее вот тут - <http://snowball.tartarus.org/algorithms/russian/stemmer.html>  
Почему он так называется? Так назывался язык программирования, который Портер написал для стеммеров. Язык так называется в созвучие языку SNOBOL. Вот комментарий самого Портера:

`Since it effectively provides a ‘suffix STRIPPER GRAMmar’, I had toyed with the idea of calling it ‘strippergram’, but good sense has prevailed, and so it is ‘Snowball’ named as a tribute to SNOBOL, the excellent string handling language of Messrs Farber, Griswold, Poage and Polonsky from the 1960s.`

Готовые стеммеры для разных языков есть в nltk. Работают они вот так:

In [53]:
from nltk.stem.snowball import SnowballStemmer

In [54]:
stemmer = SnowballStemmer('russian')

In [55]:
[(word, stemmer.stem(word)) for word in word_tokenize(text)][:30]

[('Сегодняшняя', 'сегодняшн'),
 ('заметка', 'заметк'),
 ('будет', 'будет'),
 ('о', 'о'),
 ('сервисе', 'сервис'),
 ('отслеживания', 'отслеживан'),
 ('активности', 'активн'),
 ('пользователя', 'пользовател'),
 ('—', '—'),
 ('CrazyEgg', 'CrazyEgg'),
 ('.', '.'),
 ('Я', 'я'),
 ('не', 'не'),
 ('знаю', 'зна'),
 ('кому', 'ком'),
 ('обязан', 'обяза'),
 ('сервис', 'сервис'),
 ('таким', 'так'),
 ('говорящим', 'говоря'),
 ('именем', 'имен'),
 (',', ','),
 ('но', 'но'),
 ('оно', 'он'),
 ('работает', 'работа'),
 (',', ','),
 ('и', 'и'),
 ('хорошо', 'хорош'),
 ('.', '.'),
 ('Запоминается', 'запомина'),
 ('?', '?')]

Недостатки стемминга достаточно очевидные:  
1) с супплетивными формами или редкими окончаниями слова стемминг работать не умеет  
2) к одной основе могут приводится разные слова  
3) к разным основам могут сводиться формы одного слова  
4) приставки не отбрасываются

In [60]:
stemmer.stem("пролить")

'прол'

In [61]:
stemmer.stem("прольёт")

'прольет'

In [62]:
stemmer.stem("пролом")

'прол'

# Лемматизация

Лемматизация - это замена словоформы слова в парадигме на какую-то заранее выбранную стадартную форму (лемму). 



Например, для разных форм глагола леммой обычно является неопределенная форма, а для существительного форма мужского рода единственного числа. Это позволяет избавиться от недостатков стемминга (будет, был - одна лемма), (пролить, пролом - разные). Однако лемматизация значительно сложнее. 

\* - (Значения слов "слово", "слоформа", "парадигма" приблизительно соответствует тому, которое использует Зализняк вот тут - http://inslav.ru/images/stories/pdf/2002_Zalizniak_RIS_i_statji.pdf (стр. 21-22)) Но это на самом деле не важно

К счастью есть готовые хорошие лемматизаторы. Для русского основых варианта два: Mystem и Pymorphy.


### Mystem


Майстем работает немного лучше и сам токенизирует,
поэтому можно в него засовывать сырой текст.

In [159]:
# mystem.lemmatize функция лемматизации в майстеме
# сам объект mystem нужно заранее инициализировать
# мы сделали это в начале тетрадки строчкой "mystem = Mystem()"
mystem.lemmatize(text)[:10]

['\n', 'сегодняшний', ' ', 'заметка', ' ', 'быть', ' ', 'о', ' ', 'сервис']

In [65]:
t = 'Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.'

In [66]:
# Если нужна грамматическая информация или надо сохранить ненормализованный текст,
# есть функция mystem.analyze
words_analized = mystem.analyze(text)

In [115]:
# возвращает она список словарей
# каждый словарь имеет либо одно поле 'text' (когда попался пробел) или text и analysis
# в analysis снова список словарей с вариантами разбора (первый самый вероятный)
# поля в analysis - 'gr' - грамматическая информация, 'lex' - лемма
# analysis - может быть пустым списком
words_analized[:10]

[[Parse(word='сегодняшняя', tag=OpencorporaTag('ADJF femn,sing,nomn'), normal_form='сегодняшний', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'сегодняшняя', 382, 7),))],
 [Parse(word='заметка', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='заметка', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'заметка', 8, 0),))],
 [Parse(word='будет', tag=OpencorporaTag('VERB,impf,intr sing,3per,futr,indc'), normal_form='быть', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'будет', 587, 12),))],
 [Parse(word='о', tag=OpencorporaTag('PREP'), normal_form='о', score=0.990985, methods_stack=((<DictionaryAnalyzer>, 'о', 2133, 0),)),
  Parse(word='о', tag=OpencorporaTag('INTJ'), normal_form='о', score=0.009014, methods_stack=((<DictionaryAnalyzer>, 'о', 21, 0),))],
 [Parse(word='сервисе', tag=OpencorporaTag('NOUN,inan,masc sing,loct'), normal_form='сервис', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'сервисе', 33, 5),))],
 [Parse(word='отслеживания', tag=OpencorporaTag('NOU

In [69]:
print('Слово - ', words_analized[1]['text'])
print('Разбор слова - ', words_analized[1]['analysis'][0])
print('Лемма слова - ', words_analized[1]['analysis'][0]['lex'])
print('Грамматическая информация слова - ', words_analized[1]['analysis'][0]['gr'])

Слово -  Сегодняшняя
Разбор слова -  {'lex': 'сегодняшний', 'wt': 1, 'gr': 'A=им,ед,полн,жен'}
Лемма слова -  сегодняшний
Грамматическая информация слова -  A=им,ед,полн,жен


In [1137]:
#леммы можно достать в одну строчку
[parse['analysis'][0]['lex'] for parse in words_analized if parse.get('analysis')][:10]

['сегодняшний',
 'заметка',
 'быть',
 'о',
 'сервис',
 'отслеживание',
 'активность',
 'пользователь',
 'я',
 'не']

Mystem умеет разбивать текст на предложения, но через питоновский интерфейс это сделать не получится. Нужно скачать mystem отсюда - https://yandex.ru/dev/mystem/

После этого сохранить текст в файл.

In [175]:
f = open('text.txt', 'w')
f.write(text)
f.close()

Из командной строки или из питона запустить майстем на нашем файле

In [ ]:
# про параметры почитайте в ./mystem -h
os.system(' ./mystem -iscd --format json text.txt text_parsed.txt')

В целевом файле теперь лежит разобранный текст в jsonlines (json на каждой строчке)

In [70]:
t = [json.loads(line) for line in open('text_parsed.txt')]

Каждый объект в этом списке - параграф. Каждый параграф на предложения можно разбив по тегу '//s'

Ещё так вызывать майстем может понадобиться, если важна скорость.

Недостатки Mystem: это продукт Яндекса с некоторыми ограничениями на использование, больше он не развивается.

Важным достоинством Mystem является то, что он работает не с отдельными словами, а с целым предложением. При определении нужной леммы учитывается контекст, что позволяет во многих случаях разрешать омонимию.

### Pymorphy

Pymorphy - открытый и развивается (можно поучаствовать на гитхабе)

Ссылка на репозиторий: https://github.com/kmike/pymorphy2

Попробуйте сразу установить быструю версию (pip install pymorphy2[fast])

У него нет втстроенной токенизации и он расценивает всё как слово. Когда есть несколько вариантов, он выдает их с вероятностостями, которые расчитатны на корпусе со снятой неоднозначностью. Это лучше стемминга, но хуже майстема.

In [73]:
# основная функция - pymorphy.parse
words_analized = [morph.parse(token) for token in word_tokenize(text)]

In [75]:
morph.parse("печь")

[Parse(word='печь', tag=OpencorporaTag('INFN,impf,tran'), normal_form='печь', score=0.666666, methods_stack=((<DictionaryAnalyzer>, 'печь', 2352, 0),)),
 Parse(word='печь', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='печь', score=0.166666, methods_stack=((<DictionaryAnalyzer>, 'печь', 2131, 0),)),
 Parse(word='печь', tag=OpencorporaTag('NOUN,inan,femn sing,accs'), normal_form='печь', score=0.166666, methods_stack=((<DictionaryAnalyzer>, 'печь', 2131, 3),))]

In [74]:
# Она похожа на analyze в майстеме только возрващает список объектов Parse
# Первый в списке - самый вероятный разбор (у каждого есть score)
# Информация достается через атрибут (Parse.word - например)
# Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать
# части речи или другие категории
print('Первое слово - ', words_analized[0][0].word)
print('Разбор первого слова - ', words_analized[0][0])
print('Лемма первого слова - ', words_analized[0][0].normal_form)
print('Грамматическая информация первого слова - ', words_analized[0][0].tag)
print('Часть речи первого слова - ', words_analized[0][0].tag.POS)
print('Род первого слова - ', words_analized[0][0].tag.gender)
print('Число первого слова - ', words_analized[0][0].tag.number)
print('Падеж первого слова - ', words_analized[0][0].tag.case)

Первое слово -  сегодняшняя
Разбор первого слова -  Parse(word='сегодняшняя', tag=OpencorporaTag('ADJF femn,sing,nomn'), normal_form='сегодняшний', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'сегодняшняя', 382, 7),))
Лемма первого слова -  сегодняшний
Грамматическая информация первого слова -  ADJF femn,sing,nomn
Часть речи первого слова -  ADJF
Род первого слова -  femn
Число первого слова -  sing
Падеж первого слова -  nomn


## Дополнительная очистка текста

Пунктуация часто совсем не нужна и поэтому можно выбросить её заранее. Если нужно обрабатывать много текста, это может немного ускорить процесс.


In [1154]:
# Оставим только буквено-численные токены
# Это не самый лучший вариант, так как удалятся сокращения с точкой, слова через дефис
text = 'В этом случае слова вроде т.к. и по-другому не пройдут фильтр и будут удалены.'
good_tokens = [word for word in word_tokenize(text) if word.isalnum()]
[morph.parse(token)[0].normal_form for token in good_tokens]

['в',
 'это',
 'случай',
 'слово',
 'вроде',
 'и',
 'не',
 'пройти',
 'фильтр',
 'и',
 'быть',
 'удалить']

In [114]:
#можно сделать фильтр по длине
good_tokens = [word.strip(string.punctuation) for word in 
               word_tokenize(text) 
               if len(word) > 2 and len(word) < 35]

[morph.parse(token)[0].normal_form for token in good_tokens][:10]

['сегодняшний',
 'заметка',
 'быть',
 'сервис',
 'отслеживание',
 'активность',
 'пользователь',
 'crazyegg',
 'знать',
 'кома']

Можно убрать стоп-слова (предлоги, союзы, местоимения, частотные слова). Сам термин стоп-слово происходит из информационного поиска, первый раз его упомянул [Питер Лун](https://en.wikipedia.org/wiki/Hans_Peter_Luhn) в 1959.  
Удаление таких слов позволяло сократить размер индекса и не сильно испортить выдачу или даже улучшить её, поднимая релевантность документам со значимыми словами. Со временем от такой практики, в основном, отказались - память стала дешевой (и повились всякие алгоритмы для сокращения потребления памяти), а для учёта значимости придумали IDF.  

Во многих поисковых движках стоп-слова всё ещё используются. Часто их используют и в практических задачах (классификации, тематическом моделировании). 

In [120]:
# стоп-слова есть в nltk
stops = stopwords.words('russian')
print(stops)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

Список не идеальный и его можно расширять под свои задачи.

In [144]:
words_normalized = [morph.parse(token)[0].normal_form for token in good_tokens]
[word for word in words_normalized if word not in stops]

['это',
 'случай',
 'слово',
 'вроде',
 'т.к',
 'по-другому',
 'пройти',
 'фильтр',
 'удалить']

## Предобработка для других языков

Скорее всего вы будете работать в основном только с русским языком. Но знать, чем пользоваться для других на всякий случай - тоже полезно

Nltk и gensim по умолчанию адаптированы под английский язык, поэтому разбирать их еще раз не будем.

Есть еще одна библиотека, про которую стоит рассказать - [**SpaCy**](https://spacy.io/). Это многоцелевая многоязычная библиотека. Если вам понадобится серьезно работать с английским, то лучшим вариантом будет использовать SpaCy. Другие языки там тоже поддерживаются (см. документацию - ), но не настолько хорошо как английский язык. 

В SpaCy много всего и мы будем возвращаться к ней по ходу курса. Пока посмотрим на интрументы базовой предобработки.

In [195]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

You should consider upgrading via the '/Users/mnefedov/.pyenv/versions/3.6.5/bin/python3.6 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mnefedov/.pyenv/versions/3.6.5/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
     |████████████████████████████████| 14.9 MB 179 kB/s eta 0:00:01


  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.3.0-py3-none-any.whl size=14907581 sha256=8caeb2b9cc7b3677c895a68622fa05ccf82a9928018702842c7e5b0de32cdbf2
  Stored in directory: /private/var/folders/sw/k25rhtrn2psc2hztd0t4tqmh0000gn/T/pip-ephem-wheel-cache-nd3kxcnr/wheels/fe/44/0f/7270b8ec13bc290e606a3c0f52f981915b1d09d1dfc7c79088
Successfully built de-core-news-sm
You should consider upgrading via the '/Users/mnefedov/.pyenv/versions/3.6.5/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [83]:
# загружаем пайплайн для английского языка
import spacy

nlp = spacy.load("en_core_web_sm")


In [86]:

text = ("One of the most salient features of our culture is that there is so much bullshit.” "
        "These are the opening words of the short book On Bullshit, written by the philosopher Harry Frankfurt. "
        "Fifteen years after the publication of this surprise bestseller, "
        "the rapid progress of research on artificial intelligence is forcing us to reconsider our conception "
        "of bullshit as a hallmark of human speech, with troubling implications. What do philosophical "
        "reflections on bullshit have to do with algorithms? As it turns out, quite a lot."
        "In May this year the company OpenAI, co-founded by Elon Musk in 2015, introduced "
        "a new language model called GPT-3 (for “Generative Pre-trained Transformer 3”). "
        "It took the tech world by storm. On the surface, GPT-3 is like a supercharged version "
        "of the autocomplete feature on your smartphone; it can generate coherent text based on "
        "an initial input. But GPT-3’s text-generating abilities go far beyond anything your phone "
        "is capable of. It can disambiguate pronouns, translate, infer, analogize, and even perform "
        "some forms of common-sense reasoning and arithmetic. It can generate fake news articles "
        "that humans can barely detect above chance. Given a definition, it can use a made-up word "
        "in a sentence. It can rewrite a paragraph in the style of a famous author. Yes, it can "
        "write creative fiction. Or generate code for a program based on a description of its "
        "function. It can even answer queries about general knowledge. The list goes on."
        "WHO WROTE THIS?: Not long ago, hardly anyone suspected the hoax when this self-help "
        "blog post, written by GPT-3, reached the top of Hacker News, a popular news aggregation "
        "website. “You see, creative thinking is all about having fun,” GPT-3 writes. “If you spend" 
        "too much time on it, then it stops being fun and it just feels like work.”Substack"
        "GPT-3 is a marvel of engineering due to its breathtaking scale. It contains "
        "175 billion parameters (the weights in the connections between the “neurons”" 
        "or units of the network) distributed over 96 layers. It produces embeddings in a "
        "vector space with 12,288 dimensions. And it was trained on hundreds of billions "
        "of words representing a significant subset of the Internet—including the entirety "
        "of English Wikipedia, countless books, and a dizzying number of web pages. Training "
        "the final model alone is estimated to have cost around $5 million. By all accounts, "
        "GPT-3 is a behemoth. Scaling up the size of its network and training data, without "
        "fundamental improvements to the years-old architecture, was sufficient to bootstrap "
        "the model into unexpectedly remarkable performance on a range of complex tasks, out "
        "of the box. Indeed GPT-3 is capable of “few-shot,” and even, in some cases, “zero-shot,”" 
        "learning, or learning to perform a new task without being given any example of what success "
        "looks like.")

In [87]:
doc = nlp(text)

In [92]:
for sent in doc.sents: # достаем предложения
    for token in sent: # достаем токены
        print(token.text, token.lemma_, token.pos_)

One one NUM
of of ADP
the the DET
most most ADJ
salient salient NOUN
features feature NOUN
of of ADP
our -PRON- DET
culture culture NOUN
is be AUX
that that SCONJ
there there PRON
is be AUX
so so ADV
much much ADJ
bullshit bullshit NOUN
. . PUNCT
” " PUNCT
These these DET
are be AUX
the the DET
opening open VERB
words word NOUN
of of ADP
the the DET
short short ADJ
book book NOUN
On on ADP
Bullshit Bullshit PROPN
, , PUNCT
written write VERB
by by ADP
the the DET
philosopher philosopher NOUN
Harry Harry PROPN
Frankfurt Frankfurt PROPN
. . PUNCT
Fifteen fifteen NUM
years year NOUN
after after ADP
the the DET
publication publication NOUN
of of ADP
this this DET
surprise surprise NOUN
bestseller bestseller NOUN
, , PUNCT
the the DET
rapid rapid ADJ
progress progress NOUN
of of ADP
research research NOUN
on on ADP
artificial artificial ADJ
intelligence intelligence NOUN
is be AUX
forcing force VERB
us -PRON- PRON
to to PART
reconsider reconsider VERB
our -PRON- DET
conception conception NO

In [93]:
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

Noun phrases: ['the most salient features', 'our culture', 'so much bullshit', 'the opening words', 'the short book', 'Bullshit', 'the philosopher', 'Harry Frankfurt', 'the publication', 'this surprise bestseller', 'the rapid progress', 'research', 'artificial intelligence', 'us', 'our conception', 'bullshit', 'a hallmark', 'human speech', 'troubling implications', 'What', 'philosophical reflections', 'bullshit', 'algorithms', 'it', 'May', 'the company', 'co-founded by Elon Musk', 'a new language model', 'Generative', 'Pre-trained Transformer', 'It', 'the tech world', 'storm', 'the surface', 'GPT-3', 'a supercharged version', 'the autocomplete feature', 'your smartphone', 'it', 'coherent text', 'an initial input', 'GPT-3’s text-generating abilities', 'anything', 'your phone', 'It', 'pronouns', 'some forms', 'common-sense reasoning', 'It', 'fake news articles', 'humans', 'chance', 'a definition', 'it', 'a made-up word', 'a sentence', 'It', 'a paragraph', 'the style', 'a famous author', 

In [1]:
# загружаем пайплайн для немецкого языка
import spacy

nlp = spacy.load("de_core_news_sm")


In [2]:

text = ("Vor den Stadien habe ich bis jetzt zum Glück noch keine wüsten Szenen gesehen."
        "Vorstandschef Timotheus Höttges habe sich ausgesprochen optimistisch gezeigt, schrieb Analyst Robert Grindle in einer Studie vom Montag. "
        "Während der dortigen Räterepublik war er nach dem Krieg in Künstlergruppen und Ausschüssen aktiv."
        "Welches Ergebnis die Diskussion auf EU-Ebene auch letztlich bringt, wichtig ist, dass die Preisentwicklung für die"
        "Menschen verträglicher gestaltet wird“, so Gusenbauer."
        "Weitere Informationen unter www.schnippenburg.de sowie www.eisenzeithaus.de. Es gibt neue Nachrichten auf noz.de!" 
        "Jetzt die Startseite neu laden."
        "Der Initiative 'Zivilcourage', die sich jahrelang für das Denkmal in Form eines offenen " 
        "Der islamistischen Szene Thüringens wurden nach Angaben des Thüringer Innenministeriums "
        "zuletzt etwa 125 Personen zugerechnet, der salafistischen Szene etwa 75 Personen."
        "Allerdings bestand er die EMV-Prüfung nicht, weil er Radios und DVB-T-Empfänger stört."
        )

In [3]:
doc = nlp(text)

In [4]:
for sent in doc.sents: # достаем предложения
    for token in sent: # достаем токены
        print(token.text, token.lemma_, token.pos_)

Vor Vor ADP
den der DET
Stadien Stadium NOUN
habe habe AUX
ich ich PRON
bis bis ADP
jetzt jetzt ADV
zum zum ADP
Glück Glück NOUN
noch noch ADV
keine kein DET
wüsten wüst ADJ
Szenen Szene NOUN
gesehen sehen VERB
. . PUNCT
Vorstandschef Vorstandschef NOUN
Timotheus Timotheus PROPN
Höttges Höttges PROPN
habe habe AUX
sich sich PRON
ausgesprochen aussprechen ADJ
optimistisch optimistisch ADJ
gezeigt zeigen VERB
, , PUNCT
schrieb schreiben VERB
Analyst Analyst NOUN
Robert Robert PROPN
Grindle Grindle PROPN
in in ADP
einer einer DET
Studie Studie NOUN
vom vom ADP
Montag Montag NOUN
. . PUNCT
Während während SCONJ
der der DET
dortigen dortig ADJ
Räterepublik Räterepublik NOUN
war sein AUX
er ich PRON
nach nach ADP
dem der DET
Krieg Krieg NOUN
in in ADP
Künstlergruppen Künstlergruppen NOUN
und und CCONJ
Ausschüssen Ausschuß NOUN
aktiv aktiv ADJ
. . PUNCT
Welches welch DET
Ergebnis Ergebnis NOUN
die der DET
Diskussion Diskussion NOUN
auf auf ADP
EU-Ebene EU-Ebene NOUN
auch auch ADV
letztlich le

In [5]:
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

Noun phrases: ['den Stadien', 'ich', 'Glück', 'noch keine wüsten Szenen', 'Vorstandschef Timotheus Höttges', 'sich', 'Analyst Robert Grindle', 'einer Studie', 'Montag', 'der dortigen Räterepublik', 'er', 'dem Krieg', 'Künstlergruppen', 'Ausschüssen', 'Welches Ergebnis', 'die Diskussion', 'EU-Ebene', 'die Preisentwicklung', 'dieMenschen', 'Welches Ergebnis die Diskussion auf EU-Ebene auch letztlich bringt, wichtig ist, dass die Preisentwicklung für dieMenschen verträglicher gestaltet wird“, so Gusenbauer', 'Weitere Informationen', 'www.schnippenburg.de', 'www.eisenzeithaus.de', 'neue Nachrichten', 'noz.de', 'die Startseite', "Der Initiative 'Zivilcourage", 'die', 'sich', 'das Denkmal', 'Form', 'eines offenen Der islamistischen Szene', 'Thüringens', 'Angaben', 'des Thüringer Innenministeriums', 'etwa 125 Personen', 'der salafistischen Szene', 'etwa 75 Personen', 'er', 'die EMV-Prüfung', 'er', 'Radios', 'DVB-T-Empfänger']


Есть русскоязычный форк spacy - https://github.com/buriy/spacy-ru Но это конечно не полноценный spacy. 